In [1]:
# %% [markdown]
# # 📊 Gün 1: Veri Keşfi ve Hazırlık
# 
# **Hedefler**:
# 1. Criteo uplift veri setini anlamak
# 2. Treatment/Control gruplarını incelemek
# 3. Baseline metrikleri hesaplamak
# 4. Veri kalitesini kontrol etmek

# %% [markdown]
# ## 📥 1. Kütüphaneleri ve Veriyi Yükle

# %%
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Kendi modülümüz
from src.data import (
    load_criteo_sample,
    get_features_target_treatment,
    train_test_split_uplift,
    check_treatment_balance,
    calculate_baseline_metrics,
    create_toy_dataset
)

# Görselleştirme ayarları
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✅ Kütüphaneler yüklendi!")

# %% [markdown]
# ## 📦 2. Veri Setini Yükle

# %%
# Önce toy dataset ile test edelim
print("🎯 Toy Dataset Test")
print("-" * 60)

df_toy = create_toy_dataset(n_samples=5000, treatment_effect_size=0.15)
print(df_toy.head())
print(f"\nShape: {df_toy.shape}")

# %%
# Criteo veri setini yükle (eğer hazırsa)
try:
    df = load_criteo_sample("../data/criteo_sample.parquet")
    print("✅ Criteo veri seti yüklendi!")
    print(f"Shape: {df.shape}")
    print(df.head())
except FileNotFoundError:
    print("⚠️  Criteo verisi bulunamadı. Toy dataset ile devam ediyoruz.")
    df = df_toy

# %% [markdown]
# ## 🔍 3. Veri Yapısını İncele

# %%
print("📋 VERİ SETİ BİLGİLERİ")
print("=" * 60)
print(f"Satır sayısı: {len(df):,}")
print(f"Sütun sayısı: {len(df.columns)}")
print(f"Bellek kullanımı: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print("\nSütunlar:")
print(df.dtypes)

# %%
# Eksik değer kontrolü
print("\n📊 EKSİK DEĞER ANALİZİ")
print("=" * 60)
missing = df.isnull().sum()
if missing.sum() == 0:
    print("✅ Eksik değer yok!")
else:
    print(missing[missing > 0])

# %% [markdown]
# ## 🎯 4. Treatment/Control Analizi

# %%
X, y, t = get_features_target_treatment(df)

# Treatment dağılımı
check_treatment_balance(t, "Full Dataset")

# %%
# Görselleştirme
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Treatment distribution
axes[0].bar(['Control', 'Treatment'], 
            [np.sum(t==0), np.sum(t==1)],
            color=['#ff6b6b', '#4ecdc4'])
axes[0].set_title('Treatment Distribution', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Count')
for i, v in enumerate([np.sum(t==0), np.sum(t==1)]):
    axes[0].text(i, v + 50, f'{v:,}', ha='center', va='bottom', fontweight='bold')

# Outcome distribution by treatment
outcome_by_treatment = pd.DataFrame({
    'Control': [np.sum((t==0) & (y==0)), np.sum((t==0) & (y==1))],
    'Treatment': [np.sum((t==1) & (y==0)), np.sum((t==1) & (y==1))]
}, index=['No Visit', 'Visit'])

outcome_by_treatment.plot(kind='bar', ax=axes[1], color=['#ff6b6b', '#4ecdc4'])
axes[1].set_title('Outcome Distribution by Treatment', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Count')
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=0)
axes[1].legend(title='Group')

plt.tight_layout()
plt.show()

# %% [markdown]
# ## 📈 5. Baseline Metrikleri

# %%
metrics = calculate_baseline_metrics(y, t)

# %%
# Conversion rates görselleştirme
fig, ax = plt.subplots(figsize=(8, 5))

cr_control = metrics['conversion_rate_control']
cr_treatment = metrics['conversion_rate_treatment']

bars = ax.bar(['Control', 'Treatment'], 
              [cr_control, cr_treatment],
              color=['#ff6b6b', '#4ecdc4'],
              edgecolor='black',
              linewidth=2)

ax.set_ylabel('Conversion Rate', fontsize=12)
ax.set_title('Conversion Rates: Treatment vs Control', fontsize=14, fontweight='bold')
ax.set_ylim(0, max(cr_control, cr_treatment) * 1.2)

# Değerleri göster
for i, (bar, val) in enumerate(zip(bars, [cr_control, cr_treatment])):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 0.001,
            f'{val:.4f}\n({val:.2%})',
            ha='center', va='bottom', fontweight='bold', fontsize=11)

# ATE çizgisi
ax.axhline(y=cr_control, color='red', linestyle='--', alpha=0.5, label=f'ATE = {metrics["ate"]:.4f}')
ax.legend()

plt.tight_layout()
plt.show()

print(f"\n🎯 KEY INSIGHT:")
print(f"   Treatment increases conversion by {metrics['relative_uplift']:+.2%}")

# %% [markdown]
# ## 🔬 6. Feature Distribution Analizi

# %%
print("📊 ÖZELLİK İSTATİSTİKLERİ")
print("=" * 60)
print(X.describe())

# %%
# Feature distributions (ilk 6 feature)
n_features = min(6, X.shape[1])
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.ravel()

for i in range(n_features):
    feature_name = X.columns[i]
    
    # Treatment ve Control için ayrı histogramlar
    axes[i].hist(X[t==0][feature_name], bins=30, alpha=0.5, 
                 label='Control', color='#ff6b6b', density=True)
    axes[i].hist(X[t==1][feature_name], bins=30, alpha=0.5, 
                 label='Treatment', color='#4ecdc4', density=True)
    
    axes[i].set_title(f'{feature_name} Distribution', fontweight='bold')
    axes[i].set_xlabel(feature_name)
    axes[i].set_ylabel('Density')
    axes[i].legend()
    axes[i].grid(alpha=0.3)

plt.suptitle('Feature Distributions: Treatment vs Control', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# %% [markdown]
# ## ⚖️ 7. Covariate Balance Check
# 
# **Önemli**: RCT'de (Randomized Controlled Trial) treatment ve control grupları 
# özellikler açısından dengeli olmalıdır. Aksi halde, tedavi etkisini doğru ölçemeyiz.

# %%
def check_covariate_balance(X, treatment, alpha=0.05):
    """
    Her özellik için treatment/control grupları arasında t-test yap
    """
    results = []
    
    for col in X.columns:
        x_treatment = X[treatment == 1][col]
        x_control = X[treatment == 0][col]
        
        # T-test
        t_stat, p_value = stats.ttest_ind(x_treatment, x_control)
        
        # Standardized mean difference (SMD)
        mean_diff = x_treatment.mean() - x_control.mean()
        pooled_std = np.sqrt((x_treatment.var() + x_control.var()) / 2)
        smd = mean_diff / pooled_std if pooled_std > 0 else 0
        
        results.append({
            'Feature': col,
            'Mean (Treatment)': x_treatment.mean(),
            'Mean (Control)': x_control.mean(),
            'Difference': mean_diff,
            'SMD': smd,
            't-statistic': t_stat,
            'p-value': p_value,
            'Balanced': '✅' if p_value > alpha else '⚠️'
        })
    
    return pd.DataFrame(results)

# %%
balance_df = check_covariate_balance(X, t)
print("\n⚖️  COVARIATE BALANCE TEST")
print("=" * 80)
print(balance_df.to_string(index=False))

# Özet
n_balanced = (balance_df['p-value'] > 0.05).sum()
print(f"\n📊 Summary: {n_balanced}/{len(balance_df)} features are balanced (p > 0.05)")

if n_balanced == len(balance_df):
    print("✅ Perfect balance! Randomization worked well.")
else:
    print("⚠️  Some imbalance detected. Consider using propensity score weighting.")

# %%
# SMD görselleştirmesi
fig, ax = plt.subplots(figsize=(10, 6))

colors = ['green' if abs(smd) < 0.1 else 'orange' if abs(smd) < 0.2 else 'red' 
          for smd in balance_df['SMD']]

ax.barh(balance_df['Feature'], balance_df['SMD'], color=colors, alpha=0.7, edgecolor='black')
ax.axvline(x=0, color='black', linestyle='-', linewidth=1)
ax.axvline(x=0.1, color='orange', linestyle='--', alpha=0.5, label='SMD = 0.1')
ax.axvline(x=-0.1, color='orange', linestyle='--', alpha=0.5)
ax.axvline(x=0.2, color='red', linestyle='--', alpha=0.5, label='SMD = 0.2')
ax.axvline(x=-0.2, color='red', linestyle='--', alpha=0.5)

ax.set_xlabel('Standardized Mean Difference (SMD)', fontsize=12, fontweight='bold')
ax.set_title('Covariate Balance: SMD Plot', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Interpretation:")
print("   |SMD| < 0.1: Excellent balance")
print("   |SMD| < 0.2: Acceptable balance")
print("   |SMD| ≥ 0.2: Concerning imbalance")

# %% [markdown]
# ## 📊 8. Outcome Analysis by Feature Quantiles
# 
# Özelliklerin outcome üzerindeki etkisini görmek için quantile analizi yapalım.

# %%
def analyze_feature_quantiles(X, y, treatment, feature_name, n_quantiles=5):
    """
    Bir özelliğin quantile'larına göre conversion rate analizi
    """
    feature_data = X[feature_name]
    
    # Quantile'ları oluştur
    quantiles = pd.qcut(feature_data, q=n_quantiles, labels=False, duplicates='drop')
    
    results = []
    for q in range(n_quantiles):
        mask_q = (quantiles == q)
        
        # Treatment grubu
        mask_t = mask_q & (treatment == 1)
        cr_t = y[mask_t].mean() if mask_t.sum() > 0 else np.nan
        
        # Control grubu
        mask_c = mask_q & (treatment == 0)
        cr_c = y[mask_c].mean() if mask_c.sum() > 0 else np.nan
        
        # Uplift
        uplift = cr_t - cr_c
        
        results.append({
            'Quantile': q + 1,
            'Feature Range': f"[{feature_data[mask_q].min():.2f}, {feature_data[mask_q].max():.2f}]",
            'CR (Treatment)': cr_t,
            'CR (Control)': cr_c,
            'Uplift': uplift,
            'N': mask_q.sum()
        })
    
    return pd.DataFrame(results)

# %%
# İlk feature için analiz
first_feature = X.columns[0]
quantile_df = analyze_feature_quantiles(X, y, t, first_feature, n_quantiles=5)

print(f"\n📊 QUANTILE ANALYSIS: {first_feature}")
print("=" * 80)
print(quantile_df.to_string(index=False))

# %%
# Görselleştirme
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Conversion rates
x_pos = quantile_df['Quantile']
width = 0.35

axes[0].bar(x_pos - width/2, quantile_df['CR (Control)'], width, 
            label='Control', color='#ff6b6b', alpha=0.8)
axes[0].bar(x_pos + width/2, quantile_df['CR (Treatment)'], width, 
            label='Treatment', color='#4ecdc4', alpha=0.8)

axes[0].set_xlabel('Quantile', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Conversion Rate', fontsize=12, fontweight='bold')
axes[0].set_title(f'Conversion Rates by {first_feature} Quantile', 
                  fontsize=13, fontweight='bold')
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Uplift
axes[1].bar(x_pos, quantile_df['Uplift'], color='purple', alpha=0.7, edgecolor='black')
axes[1].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[1].set_xlabel('Quantile', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Uplift', fontsize=12, fontweight='bold')
axes[1].set_title(f'Uplift by {first_feature} Quantile', 
                  fontsize=13, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n💡 INSIGHT:")
print(f"   Uplift varies across quantiles → Heterogeneous treatment effect!")
print(f"   This is why we need CATE (Conditional ATE) models like T-Learner.")

# %% [markdown]
# ## 🎯 9. Train/Test Split

# %%
X_train, X_test, y_train, y_test, t_train, t_test = train_test_split_uplift(
    df, 
    test_size=0.25, 
    random_state=42,
    stratify_treatment=True
)

print("\n📦 TRAIN/TEST SPLIT TAMAMLANDI")
print("=" * 60)
print(f"Training set size:   {len(X_train):>8,}")
print(f"Test set size:       {len(X_test):>8,}")

# %%
# Balance kontrolü
check_treatment_balance(t_train, "Training Set")
check_treatment_balance(t_test, "Test Set")

# %%
# Train ve test baseline metrikleri
print("\n🏋️ TRAINING SET METRICS")
train_metrics = calculate_baseline_metrics(y_train, t_train)

print("\n🧪 TEST SET METRICS")
test_metrics = calculate_baseline_metrics(y_test, t_test)

# Karşılaştırma
comparison = pd.DataFrame({
    'Metric': ['CR (Treatment)', 'CR (Control)', 'ATE', 'Relative Uplift'],
    'Train': [
        train_metrics['conversion_rate_treatment'],
        train_metrics['conversion_rate_control'],
        train_metrics['ate'],
        train_metrics['relative_uplift']
    ],
    'Test': [
        test_metrics['conversion_rate_treatment'],
        test_metrics['conversion_rate_control'],
        test_metrics['ate'],
        test_metrics['relative_uplift']
    ]
})

comparison['Difference'] = comparison['Test'] - comparison['Train']

print("\n📊 TRAIN vs TEST COMPARISON")
print("=" * 60)
print(comparison.to_string(index=False))

# %% [markdown]
# ## 💾 10. Veriyi Kaydet

# %%
# Eğitim için hazır veriyi kaydet
import pickle

processed_data = {
    'X_train': X_train,
    'X_test': X_test,
    'y_train': y_train,
    'y_test': y_test,
    't_train': t_train,
    't_test': t_test,
    'train_metrics': train_metrics,
    'test_metrics': test_metrics
}

output_path = '../data/processed_data.pkl'
with open(output_path, 'wb') as f:
    pickle.dump(processed_data, f)

print(f"\n✅ İşlenmiş veri kaydedildi: {output_path}")
print(f"   Boyut: {len(pickle.dumps(processed_data)) / 1024**2:.2f} MB")

# %% [markdown]
# ## 📝 11. Özet ve Sonraki Adımlar

# %%
print("\n" + "=" * 80)
print("🎉 GÜN 1 TAMAMLANDI: VERİ KEŞFİ VE HAZIRLAMA")
print("=" * 80)

print("\n✅ Tamamlanan Görevler:")
print("   1. Veri seti yüklendi ve incelendi")
print("   2. Treatment/Control dengesi kontrol edildi")
print("   3. Baseline metrikleri hesaplandı")
print("   4. Covariate balance analizi yapıldı")
print("   5. Feature-outcome ilişkileri incelendi")
print("   6. Train/test split oluşturuldu")
print("   7. İşlenmiş veri kaydedildi")

print("\n📊 Ana Bulgular:")
print(f"   • Toplam örnek sayısı: {len(df):,}")
print(f"   • Treatment ratio: {t.mean():.2%}")
print(f"   • Baseline ATE: {metrics['ate']:.4f} ({metrics['relative_uplift']:+.2%})")
print(f"   • Covariate balance: {n_balanced}/{len(balance_df)} features balanced")

print("\n🎯 Sonraki Adım (Gün 2):")
print("   → T-Learner modelini implement edin")
print("   → İki ayrı XGBoost modeli eğitin")
print("   → Uplift tahminleri oluşturun")
print("   → Notebook: 02_t_learner_basics.ipynb")

print("\n" + "=" * 80)

# %% [markdown]
# ## 🔖 Notlar ve Öğrenilenler
# 
# ### 🎓 Temel Kavramlar
# 
# 1. **Treatment Effect (Tedavi Etkisi)**:
#    - ATE = E[Y|T=1] - E[Y|T=0]
#    - Ortalama etki, ama bireysel etkiler farklı olabilir!
# 
# 2. **Randomization Check**:
#    - RCT'de gruplar dengeli olmalı
#    - Covariate balance kritik
#    - SMD < 0.1 idealdir
# 
# 3. **Heterogeneous Effects**:
#    - Uplift özellik değerlerine göre değişir
#    - Bu yüzden CATE modelleri gerekli (T-Learner, S-Learner, X-Learner)
# 
# ### 💡 Pratik İpuçları
# 
# - Treatment dengesini her zaman kontrol edin
# - Train/test split'te stratify kullanın
# - Baseline metrikleri mutlaka hesaplayın
# - Feature distributions'ı görselleştirin
# 
# ### 📚 Kaynaklar
# 
# - Radcliffe & Surry (2007): "Using control groups to target on predicted lift"
# - Künzel et al. (2019): "Metalearners for estimating heterogeneous treatment effects"
# - scikit-uplift docs: https://www.uplift-modeling.com/

# %%

✅ Kütüphaneler yüklendi!
🎯 Toy Dataset Test
------------------------------------------------------------
         f0        f1        f2        f3        f4  treatment  visit
0  0.496714 -0.138264  0.647689  1.523030 -0.234153          1      1
1 -0.234137  1.579213  0.767435 -0.469474  0.542560          1      0
2 -0.463418 -0.465730  0.241962 -1.913280 -1.724918          0      0
3 -0.562288 -1.012831  0.314247 -0.908024 -1.412304          1      1
4  1.465649 -0.225776  0.067528 -1.424748 -0.544383          0      1

Shape: (5000, 7)
⚠️  Criteo verisi bulunamadı. Toy dataset ile devam ediyoruz.
📋 VERİ SETİ BİLGİLERİ
Satır sayısı: 5,000
Sütun sayısı: 7
Bellek kullanımı: 0.23 MB

Sütunlar:
f0           float64
f1           float64
f2           float64
f3           float64
f4           float64
treatment      int32
visit          int32
dtype: object

📊 EKSİK DEĞER ANALİZİ
✅ Eksik değer yok!


KeyError: "['f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11'] not in index"